# Backpropagation
The backpropagation algorithm is useful to find the weights and biases of a Multi-Layer Perceptron (MLP). It employs the chain rule of the derivative to find the direction in which the weight values need to be modified in order to minimize the error at the output. The algorithm performs in two phases: information is first propagated forward to compute the output of the network for a given input, and the error of the network; then, the error signal is propagated backwards and the network parameters are modified accordingly.

## Loading the packages

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
from ipywidgets import interact, widgets

from matplotlib import animation

## Loading the Perceptron code
In order to the make this nothebook smaller, some of the functions (activation functions, and some of the code allowing the visualization of the results) was implemented in a separate python file. You are free to open it if needed.

In [ ]:
import mlp as mlp

## The Dataset
The following script allows you to create a 2D dataset by using the mouse. The left click adds points belonging to class A (blue), and the right click adds points belonging to class B (red). You can create as many points as you desire. The final dataset will contain hence three values per point: x coordinate (-1 ≤ x ≤ 1), y coordinate (-1 ≤ y ≤ 1) and the class ∈ {1,-1}.

In [ ]:
%matplotlib notebook

fig = pl.figure(figsize=(6,6))
pl.title("Input Dataset")
pl.xlim((-1.2,1.2))
pl.ylim((-1.2,1.2))

dataset = []

def onclick(event):
    global dataset
    cx = event.xdata
    cy = event.ydata
    co = event.button
    dataset.append((cx, cy, co-2))

    pl.scatter(cx, cy, c=(['b', 'r'])[co > 2], s=100, lw=0)
    pl.grid(True)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
fig.canvas.mpl_disconnect(onclick)

This is the dataset you just created. Check that there are no NaNs in the third column. Create once again the dataset if necessary

In [ ]:
dataset

## Finding the weights by hand
In this section you should try to find the set of weights that makes a MLP to separate the distribution of the two classes you previously defined. Use the sliders to modify the value of each one of the connections and the biases of the MLP while trying to separate the two classes (blue and red). The curve on the right represents the classification error (MSE). If the modifications you provide improve the classification, you should see the error to decrease.

In [ ]:
plotter = mlp.MLPPlotter2D(data=np.asarray(dataset))

In [ ]:
%matplotlib inline

_= interact(plotter.plot_interactive, **plotter.controls)

## The Backpropagation algorithm
Instead of trying to find the network parameters by hand, we propose to use the backpropagation algorithm. This section shows some details of its implementation. Look at the code in compute_delta_w and try to understand it.

### Derivation using the chain rule

$$ Error = \frac{1}{2} (output - target)^2 $$
$$ \Delta w = - \alpha \cdotp \frac{\partial Error}{\partial w} $$

Output layer
$$ output = f\_act(neta\_h) $$
$$ neta\_h = (w\_h\_0 \cdotp h0) + (w\_h\_1 \cdotp h1) + b_h $$

$$ \frac{\partial Error}{\partial w\_h\_i} = \frac{\partial Error}{\partial output} \cdotp \frac{\partial output}{\partial neta\_h} \cdotp \frac{\partial neta\_h}{\partial w\_h\_i} $$
$$ \frac{\partial Error}{\partial w\_h\_i} = (output - target) \cdotp f\_act'(neta\_h) \cdotp hi $$

$$ \frac{\partial Error}{\partial b\_i} = \frac{\partial Error}{\partial output} \cdotp \frac{\partial output}{\partial neta\_h} \cdotp \frac{\partial neta\_h}{\partial b\_i} $$
$$ \frac{\partial Error}{\partial b\_h} = (output - target) \cdotp f\_act'(neta\_h) \cdotp 1 $$

Hidden layer
$$ hi = f\_act(neta\_i) $$
$$ neta\_i = (x \cdotp w\_x\_i) + (y \cdotp w\_y\_i) + b_i $$

$$ \frac{\partial Error}{\partial w\_x\_i} = \frac{\partial Error}{\partial output} \cdotp \frac{\partial output}{\partial neta\_h} \cdotp \big( \frac{\partial neta\_h}{\partial h\_i} \cdotp \frac{\partial h\_i}{\partial neta\_i} \cdotp \frac{\partial neta\_i}{\partial w\_x\_i} \big) $$
$$ \frac{\partial Error}{\partial w\_x\_i} = (output - target) \cdotp f\_act'(neta\_h) \cdotp \big( w\_h\_i \cdotp f\_act'(neta\_i) \cdotp x \big) $$

$$ \frac{\partial Error}{\partial w\_y\_i} = \frac{\partial Error}{\partial output} \cdotp \frac{\partial output}{\partial neta\_h} \cdotp \big( \frac{\partial neta\_h}{\partial h\_i} \cdotp \frac{\partial h\_i}{\partial neta\_i} \cdotp \frac{\partial neta\_i}{\partial w\_y\_i} \big) $$
$$ \frac{\partial Error}{\partial w\_y\_i} = (output - target) \cdotp f\_act'(neta\_h) \cdotp \big( w\_h\_i \cdotp f\_act'(neta\_i) \cdotp y \big) $$

$$ \frac{\partial Error}{\partial b\_i} = \frac{\partial Error}{\partial output} \cdotp \frac{\partial output}{\partial neta\_h} \cdotp \big( \frac{\partial neta\_h}{\partial h\_i} \cdotp \frac{\partial h\_i}{\partial neta\_i} \frac{\partial neta\_i}{\partial b\_i} \big) $$
$$ \frac{\partial Error}{\partial b\_i} = (output - target) \cdotp f\_act'(neta\_h) \cdotp \big( w\_h\_i \cdotp f\_act'(neta\_i) \cdotp 1 \big) $$



In [ ]:
def perceptron(input_values, weights, bias, activation_function):
    neta = np.dot(input_values, weights) + bias
    return activation_function(neta)

def compute_delta_w(input_values, targets, alpha, activation_function, weights, bias):
    x = input_values[:,0]
    y = input_values[:,1]
    
    w_x_0 = weights[0]
    w_x_1 = weights[1]
    w_y_0 = weights[2]
    w_y_1 = weights[3]
    w_h_0 = weights[4]
    w_h_1 = weights[5]
    b_0   = bias[0]
    b_1   = bias[1]
    b_h   = bias[2]
    
    # feedforward
    h_0, h_0_d = perceptron(input_values, [w_x_0, w_y_0], b_0, activation_function)
    h_1, h_1_d = perceptron(input_values, [w_x_1, w_y_1], b_1, activation_function)
    h = np.array([h_0, h_1]).T
    output, output_d = perceptron(h, [w_h_0, w_h_1], b_h, activation_function)

    #output layer
    error = output - targets
    d_w_h_0 = -alpha * (error * output_d * h_0)
    d_w_h_1 = -alpha * (error * output_d * h_1)
    d_b_h   = -alpha * (error * output_d)
    
    #hidden layer
    d_w_x_0 = -alpha * (error * output_d) * (w_h_0 * h_0_d * x)
    d_w_x_1 = -alpha * (error * output_d) * (w_h_1 * h_1_d * x)
    d_w_y_0 = -alpha * (error * output_d) * (w_h_0 * h_0_d * y)
    d_w_y_1 = -alpha * (error * output_d) * (w_h_1 * h_1_d * y)
    d_b_0 = -alpha * (error * output_d) * (w_h_0 * h_0_d)
    d_b_1 = -alpha * (error * output_d) * (w_h_1 * h_1_d)
    
    return (np.vstack((d_w_x_0, d_w_x_1, d_w_y_0, d_w_y_1, d_w_h_0, d_w_h_1)).T, np.vstack((d_b_0, d_b_1, d_b_h)).T)

## Batch learning
In this section you are going to use the backpropagation algorithm to find the weights of a MLP that solves the classification problem you have previously defined. When you launch the cell, the weights and the bias are initialized at random, and the algorithm perform some iterations (NUMBER_OF_EPOCHS) doing the following:
- for each point in the dataset, compute the modifications ($\Delta w$) to be done at each parameter in order to minimize the error function
- sum up all the modifications -> batch policy
- modify the weights and biases of the MLP

The cell records the effects of the modifications performed in a video. Therefore, you can visualize the learning process afterwards.

In [ ]:
%matplotlib inline

inputs = np.asarray(dataset)[:,0:2]
targets = np.asarray(dataset)[:,2]
weights = np.random.normal(size=6)
bias = np.random.normal(size=3)
activation_function = mlp.htan

ALPHA = 0.015
NUMBER_OF_EPOCHS = 200

fig = pl.figure(figsize=(12, 4))
plotter = mlp.MLPPlotter2D(data=np.asarray(dataset))
plotter.init_animation()

def run_epoch_batch(i, alpha, inputs, targets, activation_function, weights, bias):
    d_weights, d_bias = compute_delta_w(inputs, targets, ALPHA, activation_function, weights, bias)
    weights += np.sum(d_weights, axis=0)
    bias += np.sum(d_bias, axis=0)
    
    return plotter.data2animation(i, inputs, weights, bias, targets, activation_function)

SHOW_VIDEO = True
if SHOW_VIDEO:
    anim = animation.FuncAnimation(fig, run_epoch_batch, fargs=(ALPHA, inputs, targets, activation_function, weights, bias), frames=NUMBER_OF_EPOCHS, interval=20, blit=True)
    mlp.display_animation(anim)
else:
    for i in np.arange(NUMBER_OF_EPOCHS):
        run_epoch_batch(i, ALPHA, inputs, targets, activation_function, weights, bias)


## Exercise
You are free to modify the learning rate (ALPHA) and the number of iterations (NUMBER_OF_EPOCHS).

Try different 2D classification problems and observe the behaviour of the algorithm in terms of:
- Learning rate needed
- Convergence speed
- Oscillations

Bare in mind that, in the current implementation, the parameters (weights and bias) are initialized randomly every time you launch the cell

1. What happens if the boundaries between both classes are well defined?
![separable](separable.png)

2. What happens if the classes overlap? What could you say about oscillations in the error signal?
![overlapping](overlapping.png)

3. What happens if it is not possible to separate the classes with a single line? What could you say about local minima?
![non_separable](non_separable.png)

4. What happens if the points of one of the classes are separated in subgroups (blobs)?
![blobs](blobs.png)